# Training the Model
We develop a space optimized CNN for image classification through synchronous distributed training, weight pruning, quantization, and data augmentation in Vertex AI on Google Cloud Platform. This is tested on the Cifar100 image dataset.
Testing specifications:
For data augmentation, images were shifted horizontally or vertically 10% of the time and also flipped horizontally at random. Also, the training data is normalized, which has been shown to improve accuracy. Batch size is equal to 128 and trained for 300 epochs, utilizing an exponentially decaying learning rate strategy shown in class and adam optimizer. The winning strategy was polynomial decaying of the sparsity, starting at 50% sparsity and ending at 60% sparsity. Here we finetune by trying a final sparsity of 70% instead.

# Installs

In [ ]:
!pip install tensorflow-model-optimization

In [ ]:
from __future__ import print_function
import keras
import tempfile
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import CSVLogger  
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_callbacks
import tensorflow_model_optimization as tfmot
import numpy as np
import time
import os
import pickle
from keras.datasets import cifar100
import tensorflow as tf
import multiprocessing
import os
import random
import zipfile
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Model Setup

In [ ]:
# timing per epoch iteration
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [ ]:
# using data sharding, one divide strategy for single GPU distributed training. 
# the OneDeviceStrategy places variables and computation on a single specified device.
strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")  #   tf.distribute.experimental.CentralStorageStrategy()   #tf.distribute.MirroredStrategy()

2022-12-17 06:02:21.436436: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-17 06:02:21.448143: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-17 06:02:21.448390: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-17 06:02:21.449992: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [ ]:
# define builder function for resnet layers
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [ ]:
# actual resnet code
def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)
    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
# Learning rate scheduler 
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [ ]:
# tflite conversion
def tflite_conv(model,path,quant=False):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    if quant == True:
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
    pruned_tflite_model = converter.convert()
    with open(path, 'wb') as f:
      f.write(pruned_tflite_model)
    print('Saved pruned TFLite model to:',path)

In [ ]:
# returns size of gzipped model, in bytes.
def get_gzipped_model_size(file):
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file)

    return os.path.getsize(zipped_file)

In [ ]:
# number of model replicas
strategy.num_replicas_in_sync

1

In [ ]:
# actual trainer function
def resnet_training(x_train,y_train,x_test, y_test,layers,frequency,\
                    initial_sparsity,final_sparsity,gpu="T4",\
                    const=True, poly=False,file_name='cifar10_test1',\
                    num_classes=10,begin_step=0,end_step='default'):

    # Default parameters
    batch_size = 128 * strategy.num_replicas_in_sync 
    epochs = 300
    data_augmentation = True
    n = layers

    # Computed depth from supplied model parameter n
    depth = n * 6 + 2

    # Input image dimensions.
    input_shape = x_train.shape[1:]

    # Subtracting pixel mean improves accuracy
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    print('y_train shape:', y_train.shape)

    ####Changes start#####
    num_images = x_train.shape[0] #* (1 - validation_split)

    if end_step == 'default':
        end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

    #hyperparameters: initial_sparsity=0.50, final_sparsity=0.80
    if poly:
        pruning_params = {
              'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
                                                                       final_sparsity=final_sparsity,
                                                                       begin_step=begin_step,
                                                                       end_step=end_step,
                                                                      frequency=frequency)
        }
    if const:
            pruning_params = {
              'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=final_sparsity,
                                                                       begin_step=begin_step,
                                                                       end_step=end_step,
                                                                       frequency=frequency)
        }

    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
    with strategy.scope():
        model = resnet_v1(input_shape=input_shape, depth=depth,num_classes=num_classes)
        model = prune_low_magnitude(model, **pruning_params)    #_for_pruning

        model.compile(loss= 'categorical_crossentropy',     #''  tf.  keras.losses.SparseCategoricalCrossentropy(from_logits=True)
                  optimizer=Adam(lr=lr_schedule(0)),
                  metrics=['accuracy'])
    ####Changes end#####
    model.summary()

    # Prepare model model saving directory.
    save_dir = os.path.join(os.getcwd(), 'saved_models')
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)

    model_run=file_name+'_'+str(depth)+'_'+gpu
    model_path=model_run+'.h5'
    print("RRrrrrrR",model_path)
    
    filepath = os.path.join(save_dir, model_path)
    print("RRR",filepath)
    
    print(model_run)

    # Prepare callbacks for model saving and for learning rate adjustment.
    checkpoint = ModelCheckpoint(filepath=filepath,
                                 monitor='val_acc',
                                 verbose=1,
                                 save_best_only=True)

    lr_scheduler = LearningRateScheduler(lr_schedule)

    lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                   cooldown=0,
                                   patience=5,
                                   min_lr=0.5e-6)
    logdir = tempfile.mkdtemp()
    time_callback = TimeHistory()
    logname='/home/jupyter/final_proj/log_'+model_run+'.csv'
    csv_logger = CSVLogger(logname, append=True, separator=';')
    callbacks = [checkpoint, lr_reducer, lr_scheduler, csv_logger, time_callback, pruning_callbacks.UpdatePruningStep(), tfmot.sparsity.keras.PruningSummaries(log_dir=logdir)]

    st = time.time()
    ######################## no augmentation################
    # model.fit(x_train, y_train,
    #           batch_size=batch_size,
    #           epochs=epochs,
    #           validation_data=(x_test, y_test),
    #           shuffle=True,
    #           callbacks=callbacks)
    ######################## no augmentation################

    ########################augmentation################
    datagen = ImageDataGenerator(
          # set input mean to 0 over the dataset
          featurewise_center=False,
          # set each sample mean to 0
          samplewise_center=False,
          # divide inputs by std of dataset
          featurewise_std_normalization=False,
          # divide each input by its std
          samplewise_std_normalization=False,
          # apply ZCA whitening
          zca_whitening=False,
          # epsilon for ZCA whitening
          zca_epsilon=1e-06,
          # randomly rotate images in the range (deg 0 to 180)
          rotation_range=0,
          # randomly shift images horizontally
          width_shift_range=0.1,
          # randomly shift images vertically
          height_shift_range=0.1,
          # set range for random shear
          shear_range=0.,
          # set range for random zoom
          zoom_range=0.,
          # set range for random channel shifts
          channel_shift_range=0.,
          # set mode for filling points outside the input boundaries
          fill_mode='nearest',
          # value used for fill_mode = "constant"
          cval=0.,
          # randomly flip images
          horizontal_flip=True,
          # randomly flip images
          vertical_flip=False,
          # set rescaling factor (applied before any other transformation)
          rescale=None,
          # set function that will be applied on each input
          preprocessing_function=None,
          # image data format, either "channels_first" or "channels_last"
          data_format=None,
          # fraction of images reserved for validation (strictly between 0 and 1)
          validation_split=0.0)
          
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1, 
                        callbacks=callbacks)
     
     ########################augmentation################

    %tensorboard --logdir={logdir}
    training_time = time.time() - st

    print(f"overall training time is {training_time}")
    epoch_times = time_callback.times
    print(f"each epoch training time is {epoch_times}")

    # Score trained model.
    scores = model.evaluate(x_test, y_test, verbose=1)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])
    print("tt",save_dir+'/'+model_run)
    
    #save standard model 
    model.save(save_dir+'/'+model_path)
    
    #saving data 
    with open(model_run+'.pickle', 'wb') as handle:
        pickle.dump([training_time,epoch_times,scores[0],scores[1]], handle)

    #save pruned model
    model_for_export = tfmot.sparsity.keras.strip_pruning(model)
    pruned_keras_file = save_dir+'/PRUNE_'+model_path
    keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
    print('Saved pruned Keras model to:', pruned_keras_file)
    
    #convert to tflite+ save
    pruned_tflite_file=save_dir+'/lite_PRUNE_'+model_path
    tflite_conv(model,pruned_tflite_file)
    
    #save to tf lite + qaunt
    pruned_tflite_quant_file=save_dir+'/lite_quant_PRUNE_'+model_path
    tflite_conv(model,pruned_tflite_quant_file,quant=True)
    
    pruned=get_gzipped_model_size(pruned_keras_file)
    tflite_pruned=get_gzipped_model_size(pruned_tflite_file)
    tflite_quant_pruned=get_gzipped_model_size(pruned_tflite_quant_file)
    
    
    print("Size of gzipped pruned Keras model: %.2f bytes" % (pruned))
    print("Size of gzipped pruned TFlite model: %.2f bytes" % (tflite_pruned))
    print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (tflite_quant_pruned))
    
    with open(model_run+'.pickle', 'wb') as handle:
        pickle.dump([training_time,epoch_times,scores[0],scores[1],pruned,tflite_pruned,tflite_quant_pruned], handle)

    return(model,model_path)

## Data: loading and preparation

In [ ]:
# Load the CIFAR100 data.
(X_train, Y_train), (X_test, Y_test) = cifar100.load_data()
# Normalize data.
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
# Convert class vectors to binary class matrices.
Y_train = keras.utils.to_categorical(Y_train)
Y_test = keras.utils.to_categorical(Y_test)

## Running super model with 70% final sparsity on Cifar100 with Resnet 20 and 44:

In [ ]:
layers = [3, 7]
names = ["twenty_300epochs", "forty_300epochs"]
for x in range(2):
    name="resnet_super_"+names[x]
    model1 = resnet_training(X_train,Y_train,X_test, Y_test,layers=layers[x],frequency=100,\
                    initial_sparsity = 0.5, final_sparsity=0.7,gpu="V100",\
                    const=False, poly=True,file_name=name,\
                    num_classes=100,begin_step=0,end_step='default')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 100)
Learning rate:  0.001
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 prune_low_magnitude_conv2d (Pr  (None, 32, 32, 16)  882         ['input_1[0][0]']                
 uneLowMagnitude)                                                                                 
                                                                                                  
 prune_low_magnitude_batch_norm  (None, 32, 32, 16)  65          ['prune_low_magnitude_conv2d[0][0
 alization (PruneLowMagnitude)                                   ]']  

/opt/conda/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


 prune_low_magnitude_conv2d_17   (None, 8, 8, 64)    73794       ['prune_low_magnitude_activation_
 (PruneLowMagnitude)                                             14[0][0]']                       
                                                                                                  
 prune_low_magnitude_batch_norm  (None, 8, 8, 64)    257         ['prune_low_magnitude_conv2d_17[0
 alization_15 (PruneLowMagnitud                                  ][0]']                           
 e)                                                                                               
                                                                                                  
 prune_low_magnitude_activation  (None, 8, 8, 64)    1           ['prune_low_magnitude_batch_norma
 _15 (PruneLowMagnitude)                                         lization_15[0][0]']              
                                                                                                  
 prune_low

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:153: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
2022-12-17 06:02:26.581248: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
op: "FlatMapDataset"
input: "TensorDataset/_1"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_flat_map_fn_2500"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\020FlatMapDataset:1"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
      

Learning rate:  0.001
Epoch 1/300


...

INFO:tensorflow:Assets written to: /tmp/tmpgmrzargi/assets
2022-12-17 11:57:02.634989: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-17 11:57:02.635055: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-17 11:57:02.635240: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpgmrzargi
2022-12-17 11:57:02.781953: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-17 11:57:02.782012: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpgmrzargi
2022-12-17 11:57:03.469344: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-17 11:57:05.437519: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpgmrzargi
2022-12-17 11:57:06.142703: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

Saved pruned TFLite model to: /home/jupyter/final_proj/saved_models/lite_PRUNE_resnet_super_forty_300epochs_44_V100.h5


INFO:tensorflow:Assets written to: /tmp/tmpjgm7ozc8/assets


INFO:tensorflow:Assets written to: /tmp/tmpjgm7ozc8/assets
2022-12-17 11:59:39.822166: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-17 11:59:39.822218: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-17 11:59:39.822434: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpjgm7ozc8
2022-12-17 11:59:39.971688: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-17 11:59:39.971781: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpjgm7ozc8
2022-12-17 11:59:40.691449: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-17 11:59:42.562126: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpjgm7ozc8
2022-12-17 11:59:43.247869: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

Saved pruned TFLite model to: /home/jupyter/final_proj/saved_models/lite_quant_PRUNE_resnet_super_forty_300epochs_44_V100.h5
Size of gzipped pruned Keras model: 1113673.00 bytes
Size of gzipped pruned TFlite model: 1240754.00 bytes
Size of gzipped pruned and quantized TFlite model: 1240788.00 bytes
